# 🔦 Laser Leak Detection System
**IAS Bootcamp — Laser Leak Detection Mission**

This notebook trains a YOLOv8 model to detect laser signatures on pipes.
If the laser is missing → air leakage detected.

⚠️ **Make sure GPU is enabled:** Runtime → Change runtime type → T4 GPU

## 1️⃣ Setup & Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IAS-BOOTCAMP
!ls

In [ ]:
!pip install -q ultralytics roboflow

## 2️⃣ Train the Model (YOLOv8)

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8 nano
model = YOLO('yolov8n.pt')

# Train on laser detection dataset
model.train(
    data='Laser-detection-1/data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    name='laser-leak-detector',
    project='runs',
    exist_ok=True
)

## 3️⃣ Evaluate — Confusion Matrix & Precision Report

In [ ]:
# Load best weights
model = YOLO('runs/laser-leak-detector/weights/best.pt')

# Run validation with plots
metrics = model.val(
    data='Laser-detection-1/data.yaml',
    plots=True,
    project='runs',
    name='evaluation',
    exist_ok=True
)

# Print report
print('=' * 50)
print('📊 RAPPORT D\'ÉVALUATION')
print('=' * 50)
print(f'  Précision:  {metrics.box.mp:.4f}')
print(f'  Rappel:     {metrics.box.mr:.4f}')
print(f'  mAP@50:     {metrics.box.map50:.4f}')
print(f'  mAP@50-95:  {metrics.box.map:.4f}')
f1 = 2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr + 1e-8)
print(f'  F1-Score:   {f1:.4f}')
print('=' * 50)

## 4️⃣ Visualize Results

In [ ]:
from IPython.display import Image, display

print('📊 Matrice de Confusion:')
display(Image('runs/evaluation/confusion_matrix.png'))

print('\n📈 Courbe Précision-Rappel:')
display(Image('runs/evaluation/PR_curve.png'))

print('\n📈 Courbe F1:')
display(Image('runs/evaluation/F1_curve.png'))

print('\n📈 Courbe de Précision:')
display(Image('runs/evaluation/P_curve.png'))

print('\n📈 Courbe de Rappel:')
display(Image('runs/evaluation/R_curve.png'))

## 5️⃣ Test on a Single Image

In [ ]:
import glob

# Pick a test image
test_images = glob.glob('Laser-detection-1/test/images/*')
if test_images:
    results = model.predict(test_images[0], conf=0.4, save=True, project='runs', name='predictions', exist_ok=True)
    display(Image(test_images[0]))
    print(f'\nDetections: {len(results[0].boxes)}')
    if len(results[0].boxes) == 0:
        print('🚨 ALERT: NO LASER DETECTED! Potential air leakage!')
    else:
        print('✅ Laser signature found. No leak detected.')